In [11]:
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import TextLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings, CacheBackedEmbeddings
from langchain.vectorstores import FAISS
from langchain.storage import LocalFileStore
from langchain.prompts import ChatPromptTemplate
from langchain.schema.runnable import RunnablePassthrough
from langchain.memory import ConversationBufferMemory

import nltk

nltk.download('punkt')

llm = ChatOpenAI(
    temperature=0.1,
)

cache_dir = LocalFileStore("./.cache/")

splitter = CharacterTextSplitter.from_tiktoken_encoder(
    separator="\n",
    chunk_size=600,
    chunk_overlap=100
)

loader = TextLoader("./files/chapter_one.text")

docs = loader.load_and_split(text_splitter=splitter)

embeddings = OpenAIEmbeddings()

cached_embeddings = CacheBackedEmbeddings.from_bytes_store(embeddings, cache_dir)

vectorstore = FAISS.from_documents(docs, cached_embeddings)

retriver = vectorstore.as_retriever()

memory = ConversationBufferMemory(return_messages = True)

def load_memory(_):
    x = memory.load_memory_variables({})
    return {"hitory": x["history"]}

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "Your are a helpful assistant. Answer questions using only the following contents."),
        ("human", "{question}")
    ]
)

chain = ({
    "context" : retriver,
    "question" : RunnablePassthrough(chat_history=load_memory),
}
    | prompt
    | llm 
)


chain.invoke("Is Aaronson guilty?")

AttributeError: partially initialized module 'nltk' has no attribute 'data' (most likely due to a circular import)